# checking efficiency and number of events that pass the skims

In [1]:
import ROOT

Welcome to JupyROOT 6.24/06


In [2]:
import glob

In [3]:
treename = "treevpho"

In [4]:
signal_electron_mde = glob.glob(f"./mc15_mdec_sig_e*_mc15b/sub00/*")
signal_muon_mde = glob.glob(f"./mc15_mdec_sig_mu*_mc15b/sub00/*")
signal_electron = glob.glob(f"./mc15_nomdec_sig_e*_mc15b/sub00/*")
signal_muon = glob.glob(f"./mc15_nomdec_sig_mu*_mc15b/sub00/*")

In [5]:
samples_name = [signal_electron_mde, signal_muon_mde, signal_electron,signal_muon]

In [6]:
nombre = ['signal_electron_mde', 'signal_muon_mde', 'signal_electron','signal_muon']

In [7]:
ruidos = {}
for i,n in enumerate(nombre):
    try:
        ruidos[n]={}
        ruidos[n]['df'] = ROOT.RDataFrame(treename,samples_name[i]) 
        print("cargado",n)
    except Exception as e:
        print('---->',n)
        print(e,'\n\n')
        del ruidos[n]

cargado signal_electron_mde
cargado signal_muon_mde
cargado signal_electron
cargado signal_muon


In [8]:
#print(ruidos['signal_electron_mde']['df'].GetColumnNames())

In [9]:
name_trigger=[ "no skim","passes_TauThrust", "passes_TauGeneric", "passes_TauLFV"]

In [10]:
pass_trigger = ['tau_sig_isSignal == 1 ','tau_sig_isSignal == 1 & passes_TauThrust == 1', 'tau_sig_isSignal == 1 & passes_TauGeneric == 1','tau_sig_isSignal == 1 & passes_TauLFV == 1']

In [11]:
#name_trigger=[ "no trigger", "passes_TauLFV"]

In [12]:
#pass_trigger = ['tau_sig_isSignal == 1 ','tau_sig_isSignal == 1 & passes_TauLFV == 1']

In [13]:
lista_results_eff=[]
lista_results_event=[]

In [14]:
for j,element in enumerate(pass_trigger):
    lista_trigger_eff=[]
    lista_trigger_event=[]
    lista_trigger_eff.append(name_trigger[j])
    lista_trigger_event.append(name_trigger[j])
    print(element)
    for ruido in ruidos:
        print('------------------------------------------------------------------------------------------')
        if ruido in ['signal_electron_mde','signal_electron']:
            cuts = element +' & dmID_1prong == 11'
        else:
            cuts = element +' & dmID_1prong == 13'
        count=ruidos[ruido]["df"].Filter(cuts).Count()
        print(ruido,"\t --->",count.GetValue(),"---eff->",round(count.GetValue()*100/4000000, 2))
        lista_trigger_event.append(count.GetValue())
        lista_trigger_eff.append(round(count.GetValue()*100/4000000, 2))
    lista_results_eff.append(lista_trigger_eff)
    lista_results_event.append(lista_trigger_event)

tau_sig_isSignal == 1 
------------------------------------------------------------------------------------------
signal_electron_mde 	 ---> 1180436 ---eff-> 29.51
------------------------------------------------------------------------------------------
signal_muon_mde 	 ---> 1683170 ---eff-> 42.08
------------------------------------------------------------------------------------------
signal_electron 	 ---> 1498724 ---eff-> 37.47
------------------------------------------------------------------------------------------
signal_muon 	 ---> 1764331 ---eff-> 44.11
tau_sig_isSignal == 1 & passes_TauThrust == 1
------------------------------------------------------------------------------------------
signal_electron_mde 	 ---> 1154776 ---eff-> 28.87
------------------------------------------------------------------------------------------
signal_muon_mde 	 ---> 1645646 ---eff-> 41.14
------------------------------------------------------------------------------------------
signal_electro

In [15]:
from IPython.display import HTML, display

In [16]:
def display_table1(data, caption):
    i=0
    html = "<table style='width:50%'>"
    html+= "<caption>%s</caption>" %caption
    for row in range(0,len(data[0])):
        html += "<tr>"
        for field in [data[0][i], data[1][i], data[2][i], data[3][i], data[4][i]]:
            html += "<td><h4>%s</h4><td>"%(field)
        i+=1
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

In [17]:
lista_results_event.insert(0, [' '] + nombre)
lista_results_eff.insert(0, [' '] + nombre)

In [18]:
lista_results_event

[[' ',
  'signal_electron_mde',
  'signal_muon_mde',
  'signal_electron',
  'signal_muon'],
 ['no skim', 1180436, 1683170, 1498724, 1764331],
 ['passes_TauThrust', 1154776, 1645646, 1443313, 1698708],
 ['passes_TauGeneric', 1010558, 1549165, 1287678, 1621444],
 ['passes_TauLFV', 1180436, 1683170, 1394230, 1689647]]

In [19]:
display_table1(lista_results_eff,"Efficiency")

In [20]:
display_table1(lista_results_event,"Events")

In [21]:
def MakeTLegend(xpos=0.225, ypos=0.9, size='', title='', tobjects=[], labels=[], options=[]):

    entries = len(tobjects)
    if title != '':
        entries = entries + 1

    xl = xpos
    width = 0.25
    xh = xl + width
    yh = ypos
    yl = yh - entries*0.05

    leg = ROOT.TLegend(xl, yl, xh, yh)
    leg.SetTextFont(42)
    leg.SetLineColor(0)
    leg.SetBorderSize(0)
    leg.SetFillColor(0)
    leg.SetFillStyle(0)
    if size!='':
        leg.SetTextSize(0.04)
    
    if title!= '':
        leg.SetHeader(title)

    for i, tobject in enumerate(tobjects):
        leg.AddEntry(tobject, labels[i], options[i])

    return leg

In [22]:
pass_trigger

['tau_sig_isSignal == 1 ',
 'tau_sig_isSignal == 1 & passes_TauThrust == 1',
 'tau_sig_isSignal == 1 & passes_TauGeneric == 1',
 'tau_sig_isSignal == 1 & passes_TauLFV == 1']

In [23]:
for ruido in ruidos:
    list_histos=[]
    print('--->',ruido)
    for j,element in enumerate(pass_trigger):
        print(element)
        if ruido in ['signal_electron_mde','signal_electron']:
            cuts = element +' & dmID_1prong == 11'
        else:
            cuts = element +' & dmID_1prong == 13'
        h = ruidos[ruido]['df'].Filter(cuts).Histo1D(("mass", "mass", 50, 0, 3), "tau_sig_InvM" )#"tau_sig_deltaE") #"tau_sig_InvM"
        #h = ruidos[ruido]['df'].Filter(cuts).Histo2D(("h2sig","h2sig", 20, 0, 3, 20, -2, 1), "tau_sig_InvM", "tau_sig_deltaE")
        list_histos.append(h)
    histo=[list_histos[i].GetValue() for i in range(0,len(list_histos))]
    l = MakeTLegend(xpos=0.2, ypos=0.9,tobjects=histo, labels=name_trigger, options=['f' for k in range(0, len(name_trigger))])
    canvas = ROOT.TCanvas("a","a", 1200,1200)
    histo[0].SetTitle(ruido)
    histo[0].Draw(" ")
    histo[0].SetLineWidth(2)
    histo[0].SetLineColor(ROOT.kBlack)
    histo[0].GetXaxis().SetTitle("M_{Inv} [GeV/c^{2}]")
    #histo[0].GetXaxis().SetTitle("#Delta E [GeV]")
    #histo[0].GetYaxis().SetTitleOffset(1.2)
    histo[0].GetXaxis().SetTitleOffset(1.2)
    histo[1].Draw('same')
    histo[1].SetLineWidth(1)
    histo[1].SetLineColor(ROOT.kPink)
    histo[2].Draw('same')
    histo[2].SetLineWidth(1)
    histo[2].SetLineColor(ROOT.kBlue)
    histo[3].Draw('same')
    histo[3].SetLineWidth(1)
    histo[3].SetLineColor(ROOT.kGreen)
    l.Draw('same')
    #canvas.SetLogy()
    histo[0].SetStats(0)
    #canvas.Draw()
    name_file =name_trigger[j]+ruido+'_m_1d_ok.png'
    print(name_file)
    canvas.SaveAs(name_file)
    #break

---> signal_electron_mde
tau_sig_isSignal == 1 
tau_sig_isSignal == 1 & passes_TauThrust == 1
tau_sig_isSignal == 1 & passes_TauGeneric == 1
tau_sig_isSignal == 1 & passes_TauLFV == 1
passes_TauLFVsignal_electron_mde_m_1d_ok.png
---> signal_muon_mde
tau_sig_isSignal == 1 
tau_sig_isSignal == 1 & passes_TauThrust == 1
tau_sig_isSignal == 1 & passes_TauGeneric == 1
tau_sig_isSignal == 1 & passes_TauLFV == 1
passes_TauLFVsignal_muon_mde_m_1d_ok.png
---> signal_electron
tau_sig_isSignal == 1 
tau_sig_isSignal == 1 & passes_TauThrust == 1
tau_sig_isSignal == 1 & passes_TauGeneric == 1
tau_sig_isSignal == 1 & passes_TauLFV == 1
passes_TauLFVsignal_electron_m_1d_ok.png
---> signal_muon
tau_sig_isSignal == 1 
tau_sig_isSignal == 1 & passes_TauThrust == 1
tau_sig_isSignal == 1 & passes_TauGeneric == 1
tau_sig_isSignal == 1 & passes_TauLFV == 1
passes_TauLFVsignal_muon_m_1d_ok.png


Warning in <TCanvas::Constructor>: Deleting canvas with same name: a
Warning in <TCanvas::Constructor>: Deleting canvas with same name: a
Warning in <TCanvas::Constructor>: Deleting canvas with same name: a
